Normalização do modelo BCS não linear

In [1]:
import matplotlib.pyplot as plt
import numpy as np
exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS

### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>



Função para retornar os valores de xc e x0 dado que<br>
$x_{min}<x<x_{max}$ <br>

Nova variável<br>

$xb=\frac{x-x0}{xc}$

Assim <br>

$x=xb\cdot xc+x0$

Fazendo com que a nova variável $0<xb<1$ <br>

In [2]:
def Fnorm(xlim):
    # Encontrar o fator de normalização
    # tal que xb=(x-x0)/xc
    # xmin<x<xmax
    # fazendo com que 0<xb<1
    x=(xlim[0],xlim[1]-xlim[0])
    return x
def normalizar(x,xnorm):
    xs=np.zeros((nx,1))
    for i in range(0,nx):
        xs[i]=(x[i]-xnorm[i,0])/xnorm[i,1]
    return xs
def desnormalizar(x,xnorm):
    xs=np.zeros((nx,1))
    for i in range(0,nx):
        xs[i]=x[i]*xnorm[i,1]+xnorm[i,0]
    return xs
def AplicaEscala(var,i):
    aux=var*xnorm[i,1]+xnorm[i,0]
    return aux
def normaliza_u(u,unorm):
    aux=np.zeros_like(u)
    for i in range(0,len(u)):
        aux[i]=(u[i]-unorm[i,0])/unorm[i,1]
    return aux

Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [3]:
# Valores máximos e mínimos para normalização
#Entradas
f_lim=(30,75); zclim=(0,100);pmlim=(1e6,2e6);
pbhlim=(100000,8.5e6); pwhlim=(2e6,5.2e6); qlim=(12/3600,55/3600)

In [4]:
pm=2e6; #Simplificando pm fixo
unorm=np.array([Fnorm(f_lim),Fnorm(zclim)])
xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim), unorm[0,:],unorm[1,:]])

xnorm
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

array([[1.00000000e+05, 8.40000000e+06],
       [2.00000000e+06, 3.20000000e+06],
       [3.33333333e-03, 1.19444444e-02],
       [3.00000000e+01, 4.50000000e+01],
       [0.00000000e+00, 1.00000000e+02]])

Definindo as variáveis simbólicas

In [5]:
# Criando simbolica
from casadi import *
nx = 5; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena


In [6]:
pbh = AplicaEscala(x[0],0)
pwh = AplicaEscala(x[1],1)
q = AplicaEscala(x[2],2)
fq = AplicaEscala(x[3],3)
zc = AplicaEscala(x[4],4)

In [7]:
# Entradas

fqref = u[0]*unorm[0,1]+unorm[0,0]; zcref = u[1]*unorm[1,1]+unorm[1,0];
pm=2e6;
#pm=(pm-xnorm[0,0])/xnorm[0,1]
pm
#pm = u[2];

2000000.0

Normalizando variáveis dependentes

In [8]:
# Calculo do HEAD e delta de press�o
q0 = q/Cq*(f0/fq);
H0 = -1.2454e6*q0**2 + 7.4959e3*q0 + 9.5970e2;
H = CH*H0*(fq/f0)**2; # Head
Pp = rho*g*H;
# Calculo da press�o de intake
F1 = 0.158*((rho*L1*q**2)/(D1*A1**2))*(mu/(rho*D1*q))**(1/4);
F2 = 0.158*((rho*L2*q**2)/(D2*A2**2))*(mu/(rho*D2*q))**(1/4);

In [9]:
P0 = -2.3599e9*q0**3 -1.8082e7*q0**2 +4.3346e6*q0 + 9.4355e4;
pin = pbh - rho*g*h1 - F1
P = Cp*P0*(fq/f0)**3; # Potencia
I = Inp*P/Pnp;      # Corrente

In [10]:
# Vazao do reservatorio e vazao da choke
qr  = PI*(pr - pbh);
qr


MX((2.32e-09*(12600000-(100000+(8400000*x[0])))))

In [11]:
qc  = Cc*(zc/100)*casadi.sign((pwh - pm))*casadi.sqrt(casadi.fabs(pwh - pm));

In [12]:
dpbhdt = b1/V1*(qr - q);
dpwhdt = b2/V2*(q - qc);
dqdt = 1/M*(pbh - pwh - rho*g*hw - F1 - F2 + Pp);

Aplicar escala nas derivadas

In [13]:
dpbhdt=dpbhdt*xnorm[0,1]
dpwhdt=dpwhdt*xnorm[1,1]
dqdt=dqdt*xnorm[2,1]

In [14]:
# Função casadi
dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt)
Eq_Estado = casadi.Function('Eq_Estado',[x,u],[dxdt],
                     ['x','u'],['dxdt'])

In [15]:
# Equações algébricas
#sea_nl = casadi.Function('sea_nl',[x,u],[y,pin,H],\
#                  ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

In [16]:
y=vertcat(pin,H);
ny = y.size1()

In [17]:
BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u)
                  # 'sea_nl': sea_nl
                   }
}

In [18]:
#% Func��o objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u']);
J = sum1(dxdt_0**2);

In [19]:
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;

In [20]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}

In [21]:
solver = nlpsol('solver', 'ipopt', MMQ, opt)

In [22]:
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
#yss=sea_nl(sol['x'],BCS['u'])

In [23]:
# Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],\
#     [sol['x'],yss[0]],\
#     ['x0','uss'],['xss','yss']);

# BCS['Estacionario'] = Estacionario;

In [24]:
#% Condição inicial normalizada
#unorm1=np.array([(1)/fnorm[1], (1)/zcnorm[1], (1)/pmnorm[1]])
#unorm2=np.array([fnorm[0], zcnorm[0], pmnorm[0]])

In [25]:
f_ss,zc_ss= (np.array([60, 70]))

In [26]:
uss = [f_ss,zc_ss]; # Entradas do estacionario

uss_n=normaliza_u(uss,unorm)
uss_n

array([0, 0])

In [27]:
#% Calculo do estacionario

#x0 = [0.2,0.5,0.5,0.5,0.5]
x0 = np.array([8311024.82175957,2990109.06207437,0.00995042241351780,50,50]);

In [28]:
x0_n=normalizar(x0,xnorm)
x0_n
uss_n=x0_n[3:]
uss_n

array([[0.44444444],
       [0.5       ]])

In [29]:
# args['lbx'][3] = uss[0]
# args['ubx'][3] = uss[0]  # bounds freq. solver
# args['lbx'][4] = uss[1]
# args['ubx'][4] = uss[1]   # bounds zc solver
#sol=solver('x0',x0, 'lbx', args['lbx'], 'ubx', args['ubx'], 'p', uss);
sol=solver(x0=x0_n,lbx=args['lbx'], ubx=args['ubx'], p=uss_n)
sol['x']


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



DM([0.977481, 0.309426, 0.554024, 0.444478, 0.500008])

In [30]:
np.array(sol['x'])

array([[0.97748125],
       [0.30942575],
       [0.55402427],
       [0.44447763],
       [0.50000779]])

In [31]:
xss = desnormalizar(sol['x'],xnorm)
[xss]

xss[2]*3600

array([35.82304351])

In [33]:
xss


array([[8.31084249e+06],
       [2.99016240e+06],
       [9.95084542e-03],
       [5.00014934e+01],
       [5.00007787e+01]])

In [32]:
x0.reshape(5,1)


array([[8.31102482e+06],
       [2.99010906e+06],
       [9.95042241e-03],
       [5.00000000e+01],
       [5.00000000e+01]])